In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用AutoML训练开始

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/get_started_automl_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/get_started_automl_training.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab上运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/get_started_automl_training.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在生产环境中使用AutoML。本教程涵盖了如何开始进行AutoML训练。

了解更多关于[AutoML训练](https://cloud.google.com/vertex-ai/docs/training-overview)的信息。

### 目标

在本教程中，您将学习如何使用`AutoML`来在`Vertex AI`中进行训练。

本教程使用以下谷歌云ML服务：

- `AutoML训练`
- `Vertex AI数据集`

执行的步骤包括：

- 训练图像模型
- 将图像模型导出为边缘模型
- 训练表格模型
- 将表格模型导出为云模型
- 训练文本模型
- 训练视频模型

### 建议

在谷歌云上进行端到端的MLOps时，以下是在何时使用AutoML的最佳实践：

* **您拥有有限数量的训练数据**

* **您希望在尝试自定义模型之前建立一个基准度量**

### 数据集

#### 图像

本教程使用的图像数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[Flowers数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。本教程中的数据集版本存储在一个公共云存储桶中。训练模型可以预测给定图像中的花卉类型，包括五种花卉：雏菊、蒲公英、玫瑰、向日葵或郁金香。

#### 表格

本教程使用的表格数据集来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)中的GSOD数据集。在本教程中，您仅使用年、月和日字段来预测日均温度值（mean_temp）。

#### 文本

本教程使用的文本数据集是来自[Kaggle Datasets](https://www.kaggle.com/ritresearch/happydb)的[Happy Moments数据集](https://www.kaggle.com/ritresearch/happydb)。本教程中的数据集版本存储在一个公共云存储桶中。

#### 视频

本教程使用的视频数据集是来自[MIT](http://cbcl.mit.edu/publications/ps/Kuehne_etal_iccv11.pdf)的[Human Motion数据集](https://todo)中的高尔夫挥杆识别部分。在本教程中，您使用的数据集版本存储在一个公共云存储桶中。训练模型可以预测高尔夫挥杆开始的帧。

费用
本教程使用谷歌云的计费组件：

- Vertex AI
- 云存储

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[云存储定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预计使用量生成费用估算。

## 安装

安装执行此笔记所需的软件包。

In [ ]:
import os

# Install the packages

! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage 

### 仅限 Colab 使用：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关指示操作。

**1. Vertex AI工作台**
* 无需操作，因为您已经进行了身份验证。

**2. 本地JupyterLab实例，取消注释并运行:**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

请查看如何在 https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上为您的服务账号授予云存储权限。

### 创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import sys

import google.cloud.aiplatform as aiplatform

### 初始化 Vertex AI SDK 用于 Python

为您的项目和相应的存储桶初始化 Vertex AI SDK 用于 Python。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## 自动机器学习训练任务

自动机器学习可用于自动训练各种图像模型类型。 自动机器学习自动化以下内容：

- 数据集预处理
- 特征工程
- 数据输入
- 模型架构选择
- 超参数调整
- 训练模型

了解有关[AutoML用户的Vertex AI](https://cloud.google.com/vertex-ai/docs/start/automl-users)的更多信息。

自动机器学习图像模型

AutoML 可以训练以下类型的图像模型：

- 分类
- 目标检测
- 分割

模型可以训练用于部署到云端，或者导出到边缘设备。

了解更多关于[AutoML 模型类型](https://cloud.google.com/vertex-ai/docs/start/automl-model-types)。

### 数据准备

对于图像数据，Vertex `Dataset` 资源有一些要求：

- 图像必须存储在 Cloud Storage 存储桶中。
- 每个图像文件必须是图像格式（PNG、JPEG、BMP 等）。
- 在您的 Cloud Storage 存储桶中必须有一个包含每个图像路径和标签的索引文件。
- 索引文件必须是 CSV 或 JSONL 格式。

了解更多关于[准备图像数据](https://cloud.google.com/vertex-ai/docs/datasets/prepare-image)。

#### CSV

对于图像分类，CSV索引文件有以下要求：

- 没有标题。
- 第一列是图像的云存储路径。
- 第二列是标签。
- 任何剩余列都是多标签图像分类的额外标签。

对于图像目标检测，CSV索引文件有以下要求：

- 没有标题。
- 第一列是图像的云存储路径。
- 第二列是标签。
- 第三/第四列是边界框的左上角。坐标已标准化，介于0和1之间。
- 第五/第六/第七列不使用，应为0。
- 第八/第九列是边界框的右下角。

##### ML_USE

每行还可以指定数据项在数据集用于训练时分割时要分配给哪个拆分；否则，数据集将被随机分割：80/10/10。

`ml_use`分配是通过在指定分配的列之前添加一列来指定的 - 作为第一列。该值可以是以下之一：训练、测试或验证。

#### JSONL

对于图像分类，JSONL索引文件具有以下要求：

- 每个数据项都是单独的JSON对象，位于单独的一行上。
- 键/值对`image_gcs_uri`是图像的Cloud Storage路径。
- 键/值对`display_name`是图像的标签。

    { 'image_gcs_uri': image, 
      'classification_annotations': 
          { 'display_name': label
          }
    }
    
对于多标签，标签被指定为`display_name`键/值对的列表：

    { 'image_gcs_uri': image, 
      'classification_annotations': [
          { 'display_name': label1
          },
          { 'display_name': labelN
          },
       ]
    }
    
对于目标检测，JSONL索引文件具有以下要求：

- 每个数据项都是单独的JSON对象，位于单独的一行上。
- 键/值对`image_gcs_uri`是图像的Cloud Storage路径。
- 键/值对`bounding_box_annotations`是一个列表：
    - `display_name`：对象的标签
    - `x_min`、`y_min`、`x_max`、`y_max`：边界框的坐标

{
  "image_gcs_uri": image,
  "bounding_box_annotations": [
    {
      "display name": label,
      "x_min": "X_MIN",
      "y_min": "Y_MIN",
      "x_max": "X_MAX",
      "y_max": "Y_MAX"
      }
    },
    {
      "displayName": "OBJECT2_LABEL",
      "x_min": "X_MIN",
      "y_min": "Y_MIN",
      "x_max": "X_MAX",
      "y_max": "Y_MAX"
    }
  ]
}


对于图像分割，JSONL索引文件具有以下要求：

- 每个数据项都是单独的JSON对象，位于单独的一行上。
- 键/值对`image_gcs_uri`是图像的Cloud Storage路径。
- 键/值对`category_mask_uri`是PNG格式掩码图像的Cloud Storage路径。
- 键/值对`annotation_spec_colors`是将掩码颜色映射到标签的列表。
  - 键/值对`display_name`是像素颜色掩码的标签。
  - 键/值对`color`是对应标签掩码的RGB标准化像素值（在0和1之间）。

    { 'image_gcs_uri': image, 
      'segmentation_annotations': { 'category_mask_uri': mask_image, 'annotation_spec_colors' : [ 
          { 'display_name': label, 'color': {"red": value, "blue": value, "green": value} }, ...
      ] 
    }
    
##### ML_USE

每个JSONL对象还可以指定将数据项分配给哪个分割以进行训练；否则，数据集将随机分割为80/10/10。

"data_item_resource_labels": {
      "aiplatform.googleapis.com/ml_use": "training|test|validation"
    }

*注*: 字典键字段也可以使用驼峰命名法。例如，'image_gcs_uri'也可以是'imageGcsUri'。

云存储培训数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

#### 快速查看您的数据

本教程使用存储在公共云存储桶中的Happy Moments数据集的一个版本，使用CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`ImageDataset`类的`create`方法为`Dataset`资源创建数据集，需要以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标注模式：
  - `single_label`：二元和多类分类。
  - `multi_label`：多标签多类分类。
  - `bounding_box`：目标检测。
  - `image_segmentation`：分割。

了解更多关于[ImageDataset](https://cloud.google.com/vertex-ai/docs/datasets/prepare-image)的信息。

In [ ]:
dataset = aiplatform.ImageDataset.create(
    display_name="flowers",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

### 创建并运行训练管线

要训练AutoML模型，您需要执行两个步骤：1）创建训练管线，2）运行管线。

#### 创建训练管线

使用`AutoMLImageTrainingJob`类创建AutoML训练管线，需要以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：训练模型的任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，指定是单标签（`False`）还是多标签（`True`）。
- `model_type`：用于部署的模型类型。
  - `CLOUD`：在Google Cloud上部署。
  - `CLOUD_HIGH_ACCURACY_1`：在Google Cloud上进行精度优化的部署。
  - `CLOUD_LOW_LATENCY_`：在Google Cloud上进行延迟优化的部署。
  - `MOBILE_TF_VERSATILE_1`：在边缘设备上部署。
  - `MOBILE_TF_HIGH_ACCURACY_1`：在边缘设备上进行精度优化的部署。
  - `MOBILE_TF_LOW_LATENCY_1`：在边缘设备上进行延迟优化的部署。
- `base_model`：（可选项）从现有`Model`资源进行迁移学习，仅支持图像分类。

实例化的对象是训练作业的有向无环图（DAG）。

In [ ]:
dag = aiplatform.AutoMLImageTrainingJob(
    display_name="flowers",
    prediction_type="classification",
    multi_label=False,
    model_type="MOBILE_TF_LOW_LATENCY_1",
    base_model=None,
)

print(dag)

#### 运行训练流水线

接下来，您可以运行创建的DAG来启动训练作业，方法是调用 `run`，并传入以下参数：

- `dataset`: 用于训练模型的 `Dataset` 资源。
- `model_display_name`: 训练模型的可读名称。
- `training_fraction_split`: 用于训练的数据集百分比。
- `test_fraction_split`: 用于测试（留存数据）的数据集百分比。
- `validation_fraction_split`: 用于验证的数据集百分比。
- `budget_milli_node_hours`: （可选）以毫节点小时为单位指定的最大训练时间（1000 = 节点小时）。
- `disable_early_stopping`: 如果为 `True`，则训练可能会在服务认为无法进一步改进模型目标测量之前完成，即使预算没有用完。

当 `run` 方法完成后将返回 `Model` 资源。

训练流水线的执行将需要超过 30 分钟。

In [ ]:
if os.getenv("IS_TESTING"):
    sys.exit(0)

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="flowers",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

## 回顾模型评估分数

在您的模型训练完成之后，您可以使用`list_model_evaluations()`方法回顾模型的评估分数。该方法将返回每个评估切片的迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

## 部署模型

接下来，部署您的模型进行在线预测。要部署模型，您需要调用 `deploy` 方法。

In [ ]:
endpoint = model.deploy()

发送一个在线预测请求

向部署的模型发送在线预测。

获取测试项目

您将从数据集中随意选择一个示例作为测试项目。不必担心该示例很可能在训练模型时使用过。您只是在看如何进行预测。

In [ ]:
test_item = !gsutil cat $IMPORT_FILE | head -n1
if len(str(test_item[0]).split(",")) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

### 进行预测

现在您的 `Model` 资源已部署到 `Endpoint` 资源，您可以通过将预测请求发送到 Endpoint 资源进行在线预测。

#### 请求

在此示例中，由于您的测试项目位于公共 Cloud 存储桶中，您需要将其复制到您的存储桶中，并使用 `Cloud 存储 SDK` 读取图像的内容。为了将测试数据传递给预测服务，您需要将字节编码为 base64，这样可以使内容在通过网络传输二进制数据时安全免受修改。

每个实例的格式为：

    { 'content': { 'b64': base64编码的字节 } }

由于 `predict()` 方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

`predict()` 调用的响应是一个 Python 字典，具有以下条目：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `displayNames`：每个类别标签的类名称。
- `confidences`：每个类别标签的预测置信度，介于 0 和 1 之间。
- `deployed_model_id`：执行预测的部署的 Model 资源的 Vertex AI 标识符。

In [ ]:
import base64

from google.cloud import storage

# Copy the test image to the Cloud storage bucket as "test.jpg"
test_image_local = "{}/test.jpg".format(BUCKET_URI)
! gsutil cp $test_item $test_image_local

# Download the test image in bytes format
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(bucket_name=BUCKET_URI[5:])
test_content = bucket.get_blob("test.jpg").download_as_bytes()

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{"content": base64.b64encode(test_content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)

另一种方法是使用[GFile](https://www.tensorflow.org/api_docs/python/tf/io/gfile/GFile)。另外，可以使用来自tensorflow-io库的[GFile](https://www.tensorflow.org/api_docs/python/tf/io/gfile/GFile)方法直接从云存储中读取数据。以下代码片段执行相同的操作：

```
import base64
import tensorflow as tf

# 使用GFile读取测试文件
with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

# 每个实例的格式应符合部署模型的预测输入模式。
instances = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)
```
然而，`tf.io.gfile.GFile`支持多个文件系统实现，包括本地文件、Google Cloud Storage（使用gs://前缀）和HDFS（使用hdfs://前缀）。

取消部署模型

当您完成预测后，您可以从“端点”资源中取消部署模型。这将取消所有计算资源，并停止对部署模型的计费。

In [ ]:
endpoint.undeploy_all()

将模型导出为Edge模型

您可以将AutoML云模型导出为`Edge`模型，然后可以自定义部署到边缘设备或本地下载。使用方法`export_model()`将模型导出到Cloud Storage，这需要以下参数：

- `artifact_destination`：用于存储SavedFormat模型工件的Cloud Storage位置。
- `export_format_id`：要保存模型格式的格式。对于AutoML云，只有一个选项：
   - `tf-saved-model`：用于部署到容器的TensorFlow SavedFormat。
   - `tflite`：用于部署到边缘或移动设备的TensorFlow Lite。
   - `edgetpu-tflite`：用于TPU的TensorFlow Lite。
   - `tf-js`：用于Web客户端的TensorFlow。
   - `coral-ml`：用于Coral设备。

- `sync`：是否同步或异步执行操作。

In [ ]:
response = model.export_model(
    artifact_destination=BUCKET_URI, export_format_id="tflite", sync=True
)

model_package = response["artifactOutputUri"]

删除模型

方法'delete()'将删除模型。

In [ ]:
model.delete()

删除数据集

方法'delete()'将删除数据集。

In [ ]:
dataset.delete()

删除终端点

'delete（）'方法将删除终端点。

In [ ]:
endpoint.delete()

AutoML表格模型

AutoML可以训练以下类型的表格模型：

- 分类
- 回归
- 预测

模型可以被训练以自动部署到云端，也可以被导出以手动部署到云端。

了解更多关于[AutoML模型类型](https://cloud.google.com/vertex-ai/docs/start/automl-model-types)。

### 数据准备

Vertex AI 的 `Dataset` 资源适用于表格数据，并对您的表格数据有一些要求。

- 必须保存在 CSV 文件或 BigQuery 表格中。

了解更多关于 [准备表格数据](https://cloud.google.com/vertex-ai/docs/datasets/prepare-tabular)。

#### CSV

对于表格模型，CSV 文件有一些要求：

- 第一行必须是标题行 -- 请注意这与图像、文本和视频不同，这些要求是没有标题行。
- 除了一列外，所有列都是特征。
- 一列是标签，您将在随后创建训练流程时指定。

##### ML_USE

每一行还可以指定在数据集为训练集分割时分配给数据项的分割；否则，数据集将被随机分割：80/10/10。

`ml_use` 分配通过在第一列之前添加一个列来指定分配方式。该值可以是：训练、测试或验证。

BigQuery培训数据的位置。

现在将变量 `IMPORT_FILE` 设置为BigQuery中数据表的位置。

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"
BQ_TABLE = "bigquery-public-data.samples.gsod"

### 创建数据集

#### BigQuery输入数据

接下来，使用`TabularDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `bq_source`：将数据项从BigQuery表导入到`Dataset`资源中。
- `labels`：用户定义的元数据。在这个例子中，您存储包含用户定义数据的Cloud Storage存储桶的位置。

了解更多关于[从BigQuery表创建TabularDataset](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_bigquery_sample-python)。

In [ ]:
dataset = aiplatform.TabularDataset.create(
    display_name="gsod",
    bq_source=[IMPORT_FILE],
    labels={"user_metadata": BUCKET_URI[5:]},
)

label_column = "mean_temp"

print(dataset.resource_name)

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "year"}},
    {"auto": {"column_name": "month"}},
    {"auto": {"column_name": "day"}},
]

label_column = "mean_temp"

### 创建并运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，2）运行该管道。

#### 创建训练管道

使用`AutoMLTabularTrainingJob`类创建一个AutoML训练管道，包括以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `optimization_prediction_type`：要为模型训练的任务类型。
  - `classification`：一个表格分类模型。
  - `regression`：一个表格回归模型。
- `column_transformations`：（可选）要应用于输入列的转换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 二元分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

In [ ]:
dag = aiplatform.AutoMLTabularTrainingJob(
    display_name="gsod",
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_transformations=TRANSFORMATIONS,
)

print(dag)

运行训练流水线

接下来，您可以通过调用`run`方法以以下参数来运行已创建的DAG以启动训练作业：

- `dataset`: 用于训练模型的`Dataset`资源。
- `model_display_name`: 受训模型的易理解名称。
- `training_fraction_split`: 用于训练的数据集百分比。
- `test_fraction_split`: 用于测试（保留数据）的数据集百分比。
- `validation_fraction_split`: 用于验证的数据集百分比。
- `target_column`: 要作为标签训练的列名。
- `budget_milli_node_hours`: （可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`: 若为`True`，服务可能认为在整体预算内无法进一步提高模型目标指标时，训练可能会在使用整体预算前完成。

完成`run`方法将返回`Model`资源。

训练流水线的执行将需时 >30分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="gsod",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column="mean_temp",
)

## 回顾模型评估分数

在模型训练完成后，您可以使用`list_model_evaluations()`方法来查看模型的评估分数。这个方法将为每个评估切片返回一个迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

## 部署模型

接下来，为在线预测部署您的模型。要部署模型，您可以调用`deploy`方法，并使用以下参数：

- `machine_type`：计算机类型。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

### 取消部署模型

当您完成预测时，您可以从`Endpoint`资源中取消部署模型。这会取消所有计算资源，并停止部署模型的计费。

In [ ]:
endpoint.undeploy_all()

## 导出为云模型

您可以将AutoML云模型导出为TensorFlow SavedFormat模型，然后可以自定义部署到云存储或本地下载。使用方法`export_model()`将模型导出到云存储，该方法接受以下参数：

- `artifact_destination`：用于存储SavedFormat模型工件的云存储位置。
- `export_format_id`：要保存模型格式为的格式。对于AutoML云，只有一个选项：
   - `tf-saved-model`：TensorFlow SavedFormat
- `sync`：是否同步或异步执行操作。

In [ ]:
response = model.export_model(
    artifact_destination=BUCKET_URI, export_format_id="tf-saved-model", sync=True
)

model_package = response["artifactOutputUri"]

删除模型

方法'delete()'将删除这个模型。

In [ ]:
model.delete()

删除数据集

方法'delete（）'将删除数据集。

In [ ]:
dataset.delete()

删除端点

方法'delete()'将删除端点。

In [ ]:
endpoint.delete()

## AutoML 文本模型

AutoML 可以训练以下类型的文本模型：

- 分类
- 情感分析
- 实体提取

了解更多关于[AutoML 模型类型](https://cloud.google.com/vertex-ai/docs/start/automl-model-types)

数据准备

Vertex AI的文本“数据集”资源对您的文本数据有一些要求。

- 文本示例必须存储在CSV或JSONL文件中。

了解更多关于[准备文本数据](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text)。

#### CSV

对于文本分类，CSV文件有一些要求：

- 没有标题。
- 第一列是文本示例或云存储路径到文本文件（.txt后缀）。
- 第二列是标签。
- 任何剩余列都是用于多标签文本分类的其他标签。

对于文本情感分析，CSV文件有一些要求：

- 没有标题。
- 第一列是文本示例或云存储路径到文本文件（.txt后缀）。
- 第二列是情感值。
- 第三列是最大可能的情感值。

##### ML_USE

每行还可以指定在数据集进行训练拆分时将数据项分配给哪个拆分；否则，数据集将被随机分割：80/10/10。

`ml_use` 分配是通过在第一列前加一个列来指定的。 可能的值是：训练、测试或验证。

#### JSONL

对于文本分类，JSONL文件有一些要求：

- 每个数据项是一个单独的JSON对象，在单独的一行上。
- 键/值对`text_gcs_uri`是文本文件的云存储路径。
- 键/值对`text_content`是指定内联文本的备用方式。
- 键/值对`display_name`是文本的标签。

{
  "classification_annotation": {
    "display_name": label
  },
  "text_content": text
}
{
  "classification_annotation": {
    "display_name": label
  },
  "text_gcs_uri": "gcs_uri_to_file"
}

对于多标签，标签被指定为一个`display_name`键/值对的列表：

       'classification_annotations': [
          { 'display_name': label1
          },
          { 'display_name': labelN
          },
       ]

对于文本情感分析，JSONL文件有一些要求：

- 每个数据项是一个单独的JSON对象，在单独的一行上。
- 键/值对`text_gcs_uri`是文本文件的云存储路径。
- 键/值对`text_content`是指定内联文本的备用方式。
- 键/值对`sentiment`是情感值，为大于0的整数值。
- 键/值对`sentiment_max`是情感的最大可能值。

{
  "sentiment_annotation": {
    "sentiment": number,
    "sentiment_max": number
  },
  "text_content": text,
}
{
  "sentiment_annotation": {
    "sentiment": number,
    "sentiment_max": number
  },
  "text_gcs_uri": "gcs_uri_to_file"
}

对于文本实体提取，JSONL文件有一些要求：

- 每个数据项是一个单独的JSON对象，在单独的一行上。
- 键/值对`text_gcs_uri`是文本文件的云存储路径。
- 键/值对`text_content`是指定内联文本的备用方式。
- 键/值对`start_offset`是文本起始的字符偏移量。
- 键/值对`end_offset`是文本结束的字符偏移量。
- 键/值对`display_name`是文本的标签。

{
    "text_segment_annotations": [
      {
        "start_offset":number,
        "end_offset":number,
        "display_name": label
      },
      ...
    ],
    "textContent": "inline_text"
}
{
    "textSegmentAnnotations": [
      {
        "start_offset": number,
        "end_offset": number,
        "displayName": label
      },
      ...
    ],
    "text_gcs_uri": "gcs_uri_to_file"
}

##### ML_USE

每个JSONL对象还可以额外指定将数据项分配给哪个拆分项，当数据集被拆分用于训练时；否则，数据集将被随机分割为80/10/10。

"data_item_resource_labels": {
      "aiplatform.googleapis.com/ml_use": "training|test|validation"
    }

*注意*：字典键字段也可以使用驼峰命名法。例如，'text_gcs_uri'也可以是'textGcsUri'。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-data/NL-classification/happiness.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的快乐时刻数据集的一个版本，使用一个CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来统计示例的数量，然后查看前几行。

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`TextDataset`类的`create`方法为`Dataset`资源创建数据集，需要以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标记模式。
  - `single_label`：二元和多类分类
  - `multi_label`：多标签多类分类
  - `sentiment`：情感分析
  - `extraction`：实体提取

了解更多关于[TextDataset](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text)。

In [ ]:
dataset = aiplatform.TextDataset.create(
    display_name="happydb",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
)

print(dataset.resource_name)

### 创建和运行训练流程

要训练一个AutoML模型，您需要执行两个步骤：1) 创建一个训练流程，2) 运行这个流程。

#### 创建训练流程

使用`AutoMLTextTrainingJob`类创建一个AutoML训练流程，需要以下参数：

- `display_name`: `TrainingJob`资源的可读名称。
- `prediction_type`: 为模型训练指定的任务类型。
  - `classification`: 文本分类模型。
  - `sentiment`: 文本情感分析模型。
  - `extraction`: 文本实体提取模型。
- `multi_label`: 如果是分类任务，指定是否是单标签(False)或多标签(True)。
- `sentiment_max`: 如果是情感分析任务，指定最大情感值。

In [ ]:
dag = aiplatform.AutoMLTextTrainingJob(
    display_name="happydb",
    prediction_type="classification",
    multi_label=False,
)

print(dag)

#### 运行训练流程

接下来，您可以运行已创建的DAG来启动训练作业，方法是调用`run`方法，使用以下参数：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：经过训练的模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集的百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集的百分比。
- `validation_fraction_split`：用于验证的数据集的百分比。

当`run`方法完成时，将返回`Model`资源。

训练流程的执行时间可能长达> 30分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="happydb",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

## 查看模型评估分数

在模型训练完成后，您可以使用`list_model_evaluations()`方法来查看模型的评估分数。该方法将返回每个评估切片的迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

部署模型

接下来，部署您的模型以进行在线预测。要部署模型，您需要调用`deploy`方法。

In [ ]:
endpoint = model.deploy()

取消模型部署

当您完成预测时，您会从`Endpoint`资源中取消部署模型。这将取消所有计算资源并结束对部署模型的计费。

In [ ]:
endpoint.undeploy_all()

#### 删除模型

方法'delete()'将删除模型。

In [ ]:
model.delete()

删除数据集

方法 'delete()' 将会删除数据集。

In [ ]:
dataset.delete()

#### 删除端点

方法 'delete()' 将删除端点。

In [ ]:
endpoint.delete()

## 自动ML视频模型

自动ML可以训练以下类型的视频模型：

- 分类
- 目标跟踪
- 动作识别

模型可以被训练用于部署至云端，也可以导出到边缘设备。

了解更多关于[AutoML模型类型](https://cloud.google.com/vertex-ai/docs/start/automl-model-types)。

### 数据准备

针对文本的 Vertex AI `Dataset` 资源有一些关于您的文本数据的要求。

- 文本示例必须存储在 CSV 或 JSONL 文件中。

了解有关[准备视频数据](https://cloud.google.com/vertex-ai/docs/datasets/prepare-video)的更多信息。

CSV

对于视频分类，CSV文件有一些要求：

- 没有标题。
- 第一列是视频文件在云存储中的路径。
- 第二列是标签。
- 第三列是要分类的视频中的开始时间（秒）。
- 第四列是要分类的视频中的结束时间（秒）。

对于多标签分类，每个标签都是单独的行条目。

对于视频目标追踪，CSV文件有一些要求：

- 没有标题。
- 第一列是视频文件在云存储中的路径。
- 第二列是标签。
- 第三列为空（空白）。
- 第四列是在视频中开始跟踪对象的时间（秒）。
- 第五到第八列是要追踪的对象的顶点。
    - x_min
    - y_min
    - x_max
    - y_max
    
对于动作识别，CSV文件有一些要求：

- 没有标题。
- 每行可以是以下四种格式之一：

VIDEO_URI, TIME_SEGMENT_START, TIME_SEGMENT_END, LABEL, ANNOTATION_FRAME_TIMESTAMP

VIDEO_URI, , , LABEL, ANNOTATION_FRAME_TIMESTAMP

VIDEO_URI, TIME_SEGMENT_START, TIME_SEGMENT_END, LABEL, ANNOTATION_SEGMENT_START, ANNOTATION_SEGMENT_END

VIDEO_URI, , , LABEL, ANNOTATION_SEGMENT_START, ANNOTATION_SEGMENT_END


ML_USE

每行还可以指定将数据项分配给哪个拆分当数据集分割用于训练；否则，数据集将被随机分割：80/10/10。

ml_use分配通过在指定分配的列前添加一个列来指定。该值可以是以下之一：training，或test。

JSONL

对于视频分类，CSV文件有一些要求：

- 每个数据项是一个单独的JSON对象，放在单独的一行上。
- 键/值对`video_gcs_uri`是文本文件的云存储路径。
- 键/值对`display_name`是文本的标签。
- 键/值对`start_time`是用于分类的开始时间（秒）。
- 键/值对`end_time`是用于分类的结束时间（秒）。


    {
        "video_gcs_uri": video,
        "time_segment_annotations": [{
            "display_name": label,
            "start_time": "segment的开始时间",
            "end_time": "segment的结束时间"
        }]
    }

对于视频对象跟踪，CSV文件有一些要求：

- 每个数据项是一个单独的JSON对象，放在单独的一行上。
- 键/值对`video_gcs_uri`是文本文件的云存储路径。

    {
        "video_gcs_uri": video,
        "temporal_bounding_box_annotations": [{
            "display_name": label,
            "x_min": "边界框的最左坐标",
            "x_max": "边界框的最右坐标",
            "y_min": "边界框的最上坐标",
            "y_max": "边界框的最下坐标",
            "time_offset": "检测到对象的时间帧"
        }]
    }

对于视频动作识别，CSV文件有一些要求：

- 每个数据项是一个单独的JSON对象，放在单独的一行上。
- 键/值对`video_gcs_uri`是文本文件的云存储路径。

    {
        "video_gcs_uri': video,
        "time_segments": [{
          "start_time": "完全注释段的开始时间",
          "end_time": "段的结束时间"}],
        "time_segment_annotations": [{
          "display_name": label,
          "start_time": "段的开始时间",
          "end_time": "段的结束时间"
        }]
    }

ML_USE

每个JSONL对象还可以指定将数据项分配给哪个拆分集，以用于训练；否则，数据集将被随机拆分：80/20。

"data_item_resource_labels": {
      "aiplatform.googleapis.com/ml_use": "training|test"
    }

*注意*：字典键字段也可以使用驼峰命名法。例如，'video_gcs_uri'也可以是'videoGcsUri'。

#### 云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为在云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://automl-video-demo-data/hmdb_split1_5classes_train_inf.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的Happy Moments数据集的一个版本，使用CSV索引文件。

首先快速查看数据。通过在CSV索引文件中计算行数（`wc -l`）来计算示例数量，然后查看前几行。

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`VideoDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：要将数据项导入到`Dataset`资源中的一个或多个数据集索引文件列表。
- `import_schema_uri`：数据项的数据标记模式。
  - `classification`：二元和多类分类
  - `object_tracking`：目标跟踪
  - `action_recognition`：动作识别

了解更多关于[VideoDataset](https://cloud.google.com/vertex-ai/docs/datasets/prepare-video)。

In [ ]:
dataset = aiplatform.VideoDataset.create(
    display_name="human_motion",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.video.classification,
)

print(dataset.resource_name)

### 创建并运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，以及2）运行管道。

#### 创建训练管道

使用`AutoMLVideoTrainingJob`类创建一个AutoML训练管道，包括以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：为模型训练指定的任务类型。
  - `classification`：视频分类模型。
  - `object_tracking`：视频目标追踪模型。
  - `action_recognition`：视频动作识别模型。

In [ ]:
dag = aiplatform.AutoMLVideoTrainingJob(
    display_name="human_motion",
    prediction_type="classification",
)

print(dag)

运行培训管道

接下来，通过调用方法`run`，使用以下参数来运行创建的DAG以启动训练作业：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：经过训练的模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（保留数据）的数据集百分比。

`run`方法完成后将返回`Model`资源。

培训管道的执行将需要超过30分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="human_motion",
    training_fraction_split=0.8,
    test_fraction_split=0.2,
)

## 检查模型评估分数

在模型训练完成后，您可以使用`list_model_evaluations()`方法来检查其评估分数。该方法将为每个评估切片返回一个迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

删除模型

方法 'delete()' 将删除该模型。

In [ ]:
model.delete()

删除数据集

方法 'delete()' 将删除数据集。

In [ ]:
dataset.delete()

# 清理

要清理此项目中使用的所有谷歌云资源，您可以 [删除用于此教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI